In [ ]:
import pandas as pd
import geopandas as gpd
from siuba import *

In [ ]:
import zipfile

In [ ]:
# ! pip install pygris

In [ ]:
import _utils
import importlib
importlib.reload(_utils)

In [ ]:
import shared_utils
from calitp_data_analysis import geography_utils, utils

# Selecting Corridors

In [ ]:
analysis_date = '2023-04-15'

In [ ]:
feeds = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date)

In [ ]:
tracts = _utils.get_tract_geoms()

In [ ]:
def trips_to_stops(trip_df, feed_list):
    st = shared_utils.gtfs_utils_v2.get_stop_times(analysis_date, feed_list, trip_df=trip_df)
    st = st >> distinct(_.stop_id, _.stop_sequence) >> collect()
    st = stops >> select(_.stop_id, _.geometry) >> inner_join(_, st, on='stop_id')
    return st

In [ ]:
def sjoin_tracts(stops_gdf, tracts_gdf, buffer_m):
    
    stops_gdf = stops_gdf.to_crs(geography_utils.CA_NAD83Albers)
    assert stops_gdf.crs == tracts_gdf.crs
    
    stops_gdf.geometry = stops_gdf.buffer(buffer_m)
    tracts_sjoined = gpd.sjoin(tracts_gdf, stops_gdf) >> distinct(_.GEOID, _keep_all=True)
    
    return tracts_sjoined

## Wilshire

In [ ]:
metro = feeds >> filter(_.name.str.contains('LA Metro Bus'))
metro

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, metro.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, metro.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips_720 = trips >> filter(_.route_short_name.isin(['720']), _.direction_id == 0)

In [ ]:
st_720 = trips_to_stops(trips_720, metro.feed_key)

In [ ]:
#  only keep W of Wil/Wstn
st_720 = st_720 >> filter(_.stop_sequence <= 11)

In [ ]:
# st_720.explore()

In [ ]:
wilshire = sjoin_tracts(st_720, tracts, 804)

In [ ]:
# wilshire.explore()

In [ ]:
# wilshire.to_file('wilshire.geojson')

In [ ]:
#  includes non-corridor vmt...

# trips_all = gpd.read_parquet('outputs/new_trips_with_uza.parquet')

# trips_all >> filter(_.GEOID.isin(wilshire_results.GEOID))

In [ ]:
wilshire_results = gpd.read_parquet('outputs/wilshire_trips_with_uza.parquet')

In [ ]:
# (wilshire_results >> select(-_.geometry)).to_csv('wilshire.csv')

In [ ]:
# utils.make_zipped_shapefile(wilshire_results, 'wilsh')

In [ ]:
wilshire_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
wilshire_results.sum()

In [ ]:
wilshire_results.describe()

## Fresno Route 1

In [ ]:
fresno = feeds >> filter(_.name.str.contains('Fresno Sch'))
fresno

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, fresno.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, fresno.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips_1 = trips >> filter(_.route_short_name.isin(['01']), _.direction_id == 0)

In [ ]:
trips_1 = trips_1 >> filter(_.trip_instance_key == 'db65a5adda0fc0a2744580354516ac68')

In [ ]:
st_1 = trips_to_stops(trips_1, fresno.feed_key)

In [ ]:
st_1 = st_1 >> filter(_.stop_sequence < 20) #  vertical portion only

In [ ]:
# st_1.explore()

In [ ]:
fresno = sjoin_tracts(st_1, tracts, 804) #  half-mile

In [ ]:
# fresno.explore()

In [ ]:
fresno.to_file('fresno.geojson')

In [ ]:
fresno_results = gpd.read_parquet('outputs/fresno_trips_with_uza.parquet')

In [ ]:
# (wilshire_results >> select(-_.geometry)).to_csv('wilshire.csv')

In [ ]:
# utils.make_zipped_shapefile(wilshire_results, 'wilsh')

In [ ]:
fresno_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
fresno_results.sum()

In [ ]:
fresno_results.describe()

## San Pablo Ave

In [ ]:
ac = feeds >> filter(_.name.str.contains('AC Transit'))
ac

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, ac.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, ac.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips_72r = trips >> filter(_.route_short_name.isin(['72R']), _.direction_id == 0)

In [ ]:
st_72r = trips_to_stops(trips_72r, ac.feed_key)

In [ ]:
# st_72r.explore()

In [ ]:
san_pablo = sjoin_tracts(st_72r, tracts, 804) #  half-mile

In [ ]:
# san_pablo.explore()

In [ ]:
san_pablo.to_file('san_pablo.geojson')

In [ ]:
san_pablo_results = gpd.read_parquet('outputs/sanpablo_trips_with_uza.parquet')

In [ ]:
# (wilshire_results >> select(-_.geometry)).to_csv('wilshire.csv')

In [ ]:
# utils.make_zipped_shapefile(wilshire_results, 'wilsh')

In [ ]:
san_pablo_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
san_pablo_results.sum()

In [ ]:
san_pablo_results.describe()

## Eureka H Street/Purple Route

In [ ]:
eureka = feeds >> filter(_.name.str.contains('Humboldt Schedule'))
eureka

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, eureka.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, eureka.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips.route_long_name.unique()

In [ ]:
trips_rainbow = trips >> filter(_.route_long_name.isin(['Rainbow Route']), _.direction_id == 0)

In [ ]:
tr

In [ ]:
# trips_1 = trips_1 >> filter(_.trip_instance_key == 'db65a5adda0fc0a2744580354516ac68')

In [ ]:
st_rainbow = trips_to_stops(trips_purple, eureka.feed_key)

In [ ]:
st_rainbow = st_rainbow >> filter(_.stop_sequence >= 35)

In [ ]:
# st_rainbow.explore()

In [ ]:
eureka = sjoin_tracts(st_rainbow, tracts, 804) #  half-mile

In [ ]:
eureka.explore()

In [ ]:
eureka.to_file('eureka.geojson')

In [ ]:
eureka_results = gpd.read_parquet('outputs/eureka_trips_with_uza.parquet')

In [ ]:
# (wilshire_results >> select(-_.geometry)).to_csv('wilshire.csv')

In [ ]:
# utils.make_zipped_shapefile(wilshire_results, 'wilsh')

In [ ]:
eureka_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
eureka_results.sum()

In [ ]:
eureka_results.describe()

# All Corridors Summary

In [ ]:
fresno_results['corridor'] = 'Fresno'
san_pablo_results['corridor'] = 'San Pablo Ave'
wilshire_results['corridor'] = 'Wilshire'
eureka_results['corridor'] = 'Eureka'

In [ ]:
all_results = pd.concat([fresno_results, san_pablo_results, wilshire_results, eureka_results])

In [ ]:
(all_results >> group_by(_.corridor)
             >> summarize(total_new_transit_trips = _.projected_new_transit_trips.sum(),
                          total_population = _.total_pop.sum(),
                          total_vmt = _.total_mi_auto.sum(),
                          p50_auto_trip_mi = _.p50_mi_auto.quantile(.5),
                          total_auto_trips = _.total_trips_auto.sum()
                         )

).to_csv('vmt_transit_corridors.csv')

## Redding Route 4

In [ ]:
redding = feeds >> filter(_.name.str.contains('Redding'))
redding

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, redding.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, redding.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips_4 = trips >> filter(_.route_short_name.isin(['4']), _.direction_id == 0)

In [ ]:
# trips_1 = trips_1 >> filter(_.trip_instance_key == 'db65a5adda0fc0a2744580354516ac68')

In [ ]:
st_4 = trips_to_stops(trips_4, redding.feed_key)

In [ ]:
# st_4.explore()

In [ ]:
redding = sjoin_tracts(st_4, tracts, 804) #  half-mile

In [ ]:
# redding.explore()

In [ ]:
redding.to_file('redding.geojson')

In [ ]:
redding_results = gpd.read_parquet('outputs/redding_trips_with_uza.parquet')

In [ ]:
# (wilshire_results >> select(-_.geometry)).to_csv('wilshire.csv')

In [ ]:
# utils.make_zipped_shapefile(wilshire_results, 'wilsh')

In [ ]:
redding_results.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
redding_results.sum()

In [ ]:
redding_results.describe()

## MST (table)

In [ ]:
mst = feeds >> filter(_.name.str.contains('Monterey'))
mst

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, mst.feed_key)

In [ ]:
stops.columns

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, mst.feed_key)

In [ ]:
trips.columns

In [ ]:
trips.route_short_name.unique()

In [ ]:
ab_trips = trips >> filter(_.route_short_name.isin(['A', 'B']), _.direction_id == 0)

In [ ]:
trips_20 = trips >> filter(_.route_short_name == '20', _.direction_id == 0)

In [ ]:
st_20 = shared_utils.gtfs_utils_v2.get_stop_times(analysis_date, mst.feed_key, trip_df=trips_20)

In [ ]:
st_20 = st_20 >> distinct(_.stop_id, _.stop_sequence) >> collect()

In [ ]:
st_20 = stops >> select(_.stop_id, _.geometry) >> inner_join(_, st_20, on='stop_id')

In [ ]:
st_20 = trips_to_stops(trips_20)

In [ ]:
#  SURF BRT area for joins...
st_20 = st_20 >> filter(_.stop_sequence <= 27)

In [ ]:
# st_20.explore()

In [ ]:
st_ab = trips_to_stops(ab_trips)

In [ ]:
# st_ab.explore()

In [ ]:
surf_corridor = pd.concat([st_20, st_ab])

In [ ]:
surf_corridor.explore()